In [ ]:
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import figure
import pandas as pd
import numpy as np
from datetime import datetime 
from matplotlib.dates import AutoDateFormatter, AutoDateLocator

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib notebook

mpl.rcParams['date.autoformatter.month'] = '%b-%Y'
mpl.rcParams['date.autoformatter.day'] = '%b-%d-%Y'

while True:
    model = input('Select a year to begin modeling (1951, 2000, 2016):')
    try:
        raw = open('SP'+model+'.csv')
        break
    except:
        print('Error - Enter a valid year')
data = csv.reader(raw)
firstline = True
init = True
while True:
    try:
        initial_investment = float(input('Enter initial investment: '))
        emergency_fund_percent = float(input('Enter emergency fund %: '))/100
        emergency_fund = initial_investment * (1 - emergency_fund_percent)
        if not 0 <=emergency_fund_percent <= 1:
            print('Error - Unacceptable %')
            continue
        monthly_inv = float(input('Enter monthly addition:'))
        break
    except:
        print('Error - Enter a number')
cost_average_portfolio = initial_investment
cost_average_basis = initial_investment
emergency_fund = initial_investment * emergency_fund_percent
portfolio = (initial_investment - emergency_fund)
cost_basis = (initial_investment - emergency_fund)
cash = emergency_fund
new_month = False
buy_at_10 = 0
buy_at_20 = 0
dips_bought = 0
cost_average_list = []
dip_buy_list = []
date_list = []
cash_list = []
emergency_fund_list = []
dip_list = []

for row in data:
    #ignore column titles
    if firstline:
        firstline = False
        continue

    #parse useful info
    date = row[0]
    date_list.append(date)
    date_fields = date.split('/')
    close = float(row[4])
    year = date_fields[2]
    month = date_fields[0]

    #set-up if it's the first day of calculations
    if init:
        yesterday_close = float(row[1])
        all_time_high = yesterday_close
        yesterday_month = month
        init = False
        buy_at_10 = (.9 * all_time_high)
        buy_at_20 = (.8 * all_time_high)
        print('\n'+ date, '- First day!', '${:,.2f} cash available' .format(cash))
        print('       - Dip-buying start value: ${:,.2f}'.format(portfolio))
        print('       - Cost-averaging start value: ${:,.2f}\n'.format(cost_average_portfolio))

    #calculate values
    percent_change = ((close / yesterday_close) -1) * 100
    cost_average_portfolio = cost_average_portfolio * ((percent_change / 100)+1)
    cost_average_list.append(cost_average_portfolio)
    portfolio = portfolio * ((percent_change / 100)+1)
    dip_buy_list.append(portfolio+cash)
    yesterday_close = close
    cash = (cash * 1.000041)
    emergency_fund = (portfolio+cash) * emergency_fund_percent
    

    #tracking the all-time-high
    if close >= all_time_high:
        all_time_high = close
        if buy_at_10 == 0:
            print('\n'+date, '- New all time high at {:,.2f} on' .format(close))
            print('        - Dips can now be bought again')
            print('        - Dip-buying value: ${:,.2f} + ${:,.2f} cash' .format(portfolio, cash))
            print('        - Cost-averaging value: ${:,.2f}\n'.format(cost_average_portfolio))
        buy_at_10 = (.9 * all_time_high)
        buy_at_20 = (.8 * all_time_high)

        #print('Portfolio value: ${:.2f}\n' .format(portfolio))
    #print('Percent change: {:.2f}%\n'.format(percent_change))

    #checking if it's a new month
    if month != yesterday_month:
        new_month = True
        
    #investing in new months
    if new_month:
        monthly_inv *= 1.002466
        bull_monthly = (monthly_inv - (emergency_fund / 10))
        cost_average_portfolio += monthly_inv
        cost_average_basis += monthly_inv
        yesterday_month = month
        new_month = False
        if cash >= emergency_fund or buy_at_10 == 0:
            portfolio += monthly_inv
            cost_basis += monthly_inv
        else:
            if bull_monthly < 0:
                print(date, '- LITTLE OR NO INVESTMENT DUE TO RECOUP - emergency fund at\
 ${:,.2f} out of ${:,.2f}\n' .format(cash, emergency_fund))
                cash += monthly_inv
                if cash > emergency_fund:
                    portfolio += (cash-emergency_fund)
                    cost_basis += (cash-emergency_fund)
                    cash = emergency_fund

            else:
                portfolio += monthly_inv - (emergency_fund / 10)
                cost_basis += monthly_inv - (emergency_fund / 10)
                cash += (emergency_fund / 10)
                if cash > emergency_fund:
                    portfolio += (cash-emergency_fund)
                    cost_basis += (cash-emergency_fund)
                    cash = emergency_fund

    #check for 10% drops and invest
    if close <= buy_at_10:
        portfolio += (cash / 2)
        cost_basis += (cash / 2)
        print(date, '- 10% dip! - Buying the dip with 1/2 of cash: ${:,.2f}' .format(cash/2))
        cash -= (cash / 2)
        buy_at_10 = 0
        dips_bought += 1
        print('       - Dip-buying value: ${:,.2f} + ${:,.2f} cash' .format(portfolio, cash))
        print('       - Cost-averaging value: ${:,.2f}\n'.format(cost_average_portfolio))
        dip_list.append(date)

    #check for 20% drops and invest
    if close <= buy_at_20:
        portfolio += cash
        cost_basis += cash
        print(date, '- 20% dip! - Buying the dip with all remaing cash: ${:,.2f}' .format(cash))
        cash -= cash
        buy_at_20 = 0
        dips_bought += 1
        print('       - Dip-buying value: ${:,.2f} + ${:,.2f} cash' .format(portfolio, cash))
        print('       - Cost-averaging value: ${:,.2f}\n'.format(cost_average_portfolio))
        dip_list.append(date)

    cash_list.append(cash)
    emergency_fund_list.append(emergency_fund)

print('\n\nSIMULATION ENDED ON:', date)
print('\nDips bought:', dips_bought)
print('\nDip-buying portfolio value: ${:,.2f} + ${:,.2f} cash' .format(portfolio, cash))
print('Dip-buying portfolio cost basis: ${:,.2f}' .format(cost_basis))
print('\nCost-averaging portfolio value: ${:,.2f}' .format(cost_average_portfolio))
print('Cost-averaging portfolio cost basis: ${:,.2f}\n' .format(cost_average_basis))


#graphing and time series setup
date_list = [datetime.strptime(date, '%m/%d/%Y') for date in date_list]
abs_max = max(max(dip_buy_list),max(cost_average_list))

fig = plt.figure(figsize = (9.5,6))
ax = plt.axes()
plt.subplots_adjust(bottom = .15)
plt.grid()
plt.ylim(0,(abs_max*1.1))
plt.xlim(date_list[0],date_list[-1])
plt.xticks(rotation=30)


#time series setup for x-axis
xtick_locator = AutoDateLocator()
xtick_formatter = AutoDateFormatter(xtick_locator)
ax.xaxis.set_major_locator(xtick_locator)
ax.xaxis.set_major_formatter(xtick_formatter)

#ax.plot(date_list, emergency_fund_list, label='Target Emergency Fund')
ax.plot(date_list, cash_list, label='Available Cash')
ax.plot(date_list, dip_buy_list, label='Dip Buying')
ax.plot(date_list, cost_average_list, label='Cost Averaging')
ax.set_xlabel('Date')
ax.set_ylabel('Dollars')
ax.set_title('Buy the Dip?')
plt.legend()

### 